<a href="https://colab.research.google.com/github/HesusG/diagnostico-lineas-accion/blob/main/Semana3/notebooks/02_analisis_datos_ong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis Exploratorio de Datos de la ONG

**Curso:** CD2001B - Diagnóstico para Líneas de Acción  
**Semana 3:** Ética, Compromiso Social y Diagnóstico Estratégico  
**Objetivo:** Realizar análisis exploratorio de los datos proporcionados por tu ONG socio formador

---

## 📋 Contenido del Notebook

1. Carga y preparación de datos
2. Análisis descriptivo básico
3. Identificación de variables clave (potenciales KPIs)
4. Visualizaciones exploratorias
5. Hallazgos preliminares
6. Preparación para Actividad #1 Parte 2

## 1. Carga y Preparación de Datos

### 1.1 Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Configuración de visualizaciones
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuración pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("✅ Librerías importadas correctamente")

### 1.2 Cargar Datos de la ONG

**IMPORTANTE:** Reemplaza la ruta con el archivo de datos que te proporcionó tu ONG.

Los datos pueden estar en diferentes formatos:
- CSV: `pd.read_csv('ruta/archivo.csv')`
- Excel: `pd.read_excel('ruta/archivo.xlsx')`
- Google Sheets: Ver guía de conexión

**Ejemplo con datos simulados:**

In [ ]:
# OPCIÓN 1: Si tienes un archivo CSV de tu ONG
# df = pd.read_csv('ruta_a_tus_datos/datos_ong.csv')

# OPCIÓN 2: Crear datos de ejemplo (elimina esto cuando tengas datos reales)
# Este ejemplo simula datos de un comedor comunitario
np.random.seed(42)
n_registros = 200

df_ejemplo = pd.DataFrame({
    'fecha': pd.date_range(start='2024-01-01', periods=n_registros, freq='D'),
    'beneficiario_id': np.arange(1, n_registros + 1),
    'edad': np.random.choice([5, 6, 7, 8, 9, 10, 11, 12], n_registros),
    'genero': np.random.choice(['F', 'M'], n_registros),
    'colonia': np.random.choice(['Centro', 'La Margarita', 'Analco', 'Los Remedios'], n_registros),
    'tipo_servicio': np.random.choice(['Desayuno', 'Comida', 'Ambos'], n_registros, p=[0.4, 0.3, 0.3]),
    'satisfaccion': np.random.randint(1, 11, n_registros),  # Escala 1-10
    'asistencia_mensual': np.random.randint(10, 31, n_registros),  # Días asistidos en el mes
    'tiempo_espera_min': np.random.randint(5, 45, n_registros),
    'calidad_comida': np.random.randint(1, 11, n_registros)
})

df = df_ejemplo  # Reemplaza esto con tus datos reales

print(f"✅ Datos cargados: {len(df)} registros, {len(df.columns)} variables")
print("\n🔍 Primeras 5 filas:")
df.head()

### 1.3 Inspección Inicial

In [ ]:
# Información general del dataset
print("INFORMACIÓN DEL DATASET")
print("="*70)
df.info()

print("\n" + "="*70)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("="*70)
df.describe()

In [ ]:
# Verificar valores faltantes
print("VALORES FALTANTES")
print("="*70)
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Valores Faltantes': missing,
    'Porcentaje (%)': missing_pct
})

missing_df = missing_df[missing_df['Valores Faltantes'] > 0].sort_values('Valores Faltantes', ascending=False)

if len(missing_df) > 0:
    print(missing_df)
    print("\n⚠️ Hay variables con valores faltantes. Considera estrategias de limpieza.")
else:
    print("✅ No hay valores faltantes en el dataset")

## 2. Análisis Descriptivo por Variable

### 2.1 Variables Demográficas

In [ ]:
# Distribución por género (ajusta según tus variables)
if 'genero' in df.columns or 'sexo' in df.columns:
    col_genero = 'genero' if 'genero' in df.columns else 'sexo'
    
    print("DISTRIBUCIÓN POR GÉNERO")
    print("="*70)
    print(df[col_genero].value_counts())
    print(f"\nPorcentajes:")
    print(df[col_genero].value_counts(normalize=True) * 100)
    
    # Visualización
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    df[col_genero].value_counts().plot(kind='bar', ax=ax1, color=['#0062A4', '#009FDA'])
    ax1.set_title('Distribución por Género', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Género')
    ax1.set_ylabel('Frecuencia')
    ax1.tick_params(axis='x', rotation=0)
    
    df[col_genero].value_counts().plot(kind='pie', ax=ax2, autopct='%1.1f%%', 
                                        colors=['#0062A4', '#009FDA'])
    ax2.set_title('Proporción por Género', fontsize=14, fontweight='bold')
    ax2.set_ylabel('')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Distribución por edad (ajusta según tus variables)
if 'edad' in df.columns:
    print("DISTRIBUCIÓN POR EDAD")
    print("="*70)
    print(df['edad'].describe())
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histograma
    ax1.hist(df['edad'], bins=20, color='#0062A4', alpha=0.7, edgecolor='black')
    ax1.axvline(df['edad'].mean(), color='#FF6F31', linestyle='--', linewidth=2, label=f'Media: {df["edad"].mean():.1f}')
    ax1.axvline(df['edad'].median(), color='#8CC63F', linestyle='--', linewidth=2, label=f'Mediana: {df["edad"].median():.1f}')
    ax1.set_title('Distribución de Edad', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Edad')
    ax1.set_ylabel('Frecuencia')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Boxplot
    ax2.boxplot(df['edad'], vert=True)
    ax2.set_title('Boxplot de Edad', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Edad')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### 2.2 Variables de Calidad del Servicio (Potenciales KPIs)

In [ ]:
# Identifica las variables que miden calidad del servicio
# Estas son candidatas a KPIs

# Ejemplo: Satisfacción
if 'satisfaccion' in df.columns:
    print("KPI POTENCIAL: SATISFACCIÓN DEL BENEFICIARIO")
    print("="*70)
    print(f"Media: {df['satisfaccion'].mean():.2f}")
    print(f"Mediana: {df['satisfaccion'].median():.2f}")
    print(f"Desviación Estándar: {df['satisfaccion'].std():.2f}")
    print(f"Mínimo: {df['satisfaccion'].min()}")
    print(f"Máximo: {df['satisfaccion'].max()}")
    
    # Clasificación por niveles
    df['nivel_satisfaccion'] = pd.cut(df['satisfaccion'], 
                                       bins=[0, 4, 7, 10], 
                                       labels=['Baja', 'Media', 'Alta'])
    
    print(f"\nDistribución por nivel:")
    print(df['nivel_satisfaccion'].value_counts())
    print(f"\nPorcentaje con satisfacción Alta (>7): {(df['satisfaccion'] > 7).sum() / len(df) * 100:.1f}%")
    
    # Visualización
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    df['satisfaccion'].hist(bins=10, ax=ax1, color='#0062A4', edgecolor='black')
    ax1.axvline(df['satisfaccion'].mean(), color='#FF6F31', linestyle='--', linewidth=2, 
                label=f'Media: {df["satisfaccion"].mean():.2f}')
    ax1.set_title('Distribución de Satisfacción', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Nivel de Satisfacción (1-10)')
    ax1.set_ylabel('Frecuencia')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    df['nivel_satisfaccion'].value_counts().plot(kind='bar', ax=ax2, 
                                                  color=['#FF6F31', '#939598', '#8CC63F'])
    ax2.set_title('Satisfacción por Nivel', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Nivel')
    ax2.set_ylabel('Frecuencia')
    ax2.tick_params(axis='x', rotation=0)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Ejemplo: Tiempo de espera
if 'tiempo_espera_min' in df.columns or 'tiempo_espera' in df.columns:
    col_tiempo = 'tiempo_espera_min' if 'tiempo_espera_min' in df.columns else 'tiempo_espera'
    
    print("KPI POTENCIAL: TIEMPO DE ESPERA")
    print("="*70)
    print(f"Media: {df[col_tiempo].mean():.2f} minutos")
    print(f"Mediana: {df[col_tiempo].median():.2f} minutos")
    print(f"Desviación Estándar: {df[col_tiempo].std():.2f} minutos")
    print(f"\nPorcentaje con espera < 15 min: {(df[col_tiempo] < 15).sum() / len(df) * 100:.1f}%")
    print(f"Porcentaje con espera > 30 min: {(df[col_tiempo] > 30).sum() / len(df) * 100:.1f}%")
    
    # Visualización
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(df[col_tiempo], bins=20, color='#009FDA', alpha=0.7, edgecolor='black')
    plt.axvline(df[col_tiempo].mean(), color='#FF6F31', linestyle='--', linewidth=2, 
                label=f'Media: {df[col_tiempo].mean():.1f} min')
    plt.axvline(15, color='#8CC63F', linestyle=':', linewidth=2, label='Meta: 15 min')
    plt.title('Distribución de Tiempo de Espera', fontsize=14, fontweight='bold')
    plt.xlabel('Tiempo (minutos)')
    plt.ylabel('Frecuencia')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 2, 2)
    plt.boxplot(df[col_tiempo])
    plt.title('Boxplot de Tiempo de Espera', fontsize=14, fontweight='bold')
    plt.ylabel('Tiempo (minutos)')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 3. Análisis Cruzado

### 3.1 Satisfacción por Grupo Demográfico

In [ ]:
# Satisfacción por género
if 'satisfaccion' in df.columns and ('genero' in df.columns or 'sexo' in df.columns):
    col_genero = 'genero' if 'genero' in df.columns else 'sexo'
    
    print("SATISFACCIÓN POR GÉNERO")
    print("="*70)
    satisfaccion_genero = df.groupby(col_genero)['satisfaccion'].agg(['mean', 'median', 'std', 'count'])
    print(satisfaccion_genero)
    
    # Prueba t para ver si hay diferencia significativa
    grupos = df[col_genero].unique()
    if len(grupos) == 2:
        grupo1 = df[df[col_genero] == grupos[0]]['satisfaccion']
        grupo2 = df[df[col_genero] == grupos[1]]['satisfaccion']
        t_stat, p_value = stats.ttest_ind(grupo1, grupo2)
        
        print(f"\nPrueba t independiente:")
        print(f"t = {t_stat:.3f}, p-valor = {p_value:.4f}")
        if p_value < 0.05:
            print("✅ Hay diferencia SIGNIFICATIVA entre grupos (p < 0.05)")
        else:
            print("❌ NO hay diferencia significativa entre grupos (p >= 0.05)")
    
    # Visualización
    plt.figure(figsize=(10, 6))
    df.boxplot(column='satisfaccion', by=col_genero, figsize=(8, 6))
    plt.title('Satisfacción por Género', fontsize=14, fontweight='bold')
    plt.suptitle('')  # Remove auto title
    plt.xlabel('Género')
    plt.ylabel('Satisfacción (1-10)')
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
# Satisfacción por ubicación geográfica
if 'satisfaccion' in df.columns and ('colonia' in df.columns or 'zona' in df.columns or 'municipio' in df.columns):
    col_ubicacion = next((col for col in ['colonia', 'zona', 'municipio'] if col in df.columns), None)
    
    if col_ubicacion:
        print(f"SATISFACCIÓN POR {col_ubicacion.upper()}")
        print("="*70)
        satisfaccion_ubicacion = df.groupby(col_ubicacion)['satisfaccion'].agg(['mean', 'median', 'std', 'count'])
        satisfaccion_ubicacion = satisfaccion_ubicacion.sort_values('mean', ascending=False)
        print(satisfaccion_ubicacion)
        
        # Visualización
        plt.figure(figsize=(12, 6))
        satisfaccion_ubicacion['mean'].plot(kind='bar', color='#0062A4', alpha=0.7)
        plt.axhline(df['satisfaccion'].mean(), color='#FF6F31', linestyle='--', linewidth=2, 
                    label=f'Media general: {df["satisfaccion"].mean():.2f}')
        plt.title(f'Satisfacción Promedio por {col_ubicacion.title()}', fontsize=14, fontweight='bold')
        plt.xlabel(col_ubicacion.title())
        plt.ylabel('Satisfacción Promedio')
        plt.legend()
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.show()

### 3.2 Correlaciones entre Variables

In [ ]:
# Matriz de correlación para variables numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Excluir IDs y fechas si existen
exclude_cols = [col for col in numeric_cols if 'id' in col.lower() or 'fecha' in col.lower()]
numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

if len(numeric_cols) >= 2:
    print("MATRIZ DE CORRELACIÓN")
    print("="*70)
    
    correlation_matrix = df[numeric_cols].corr()
    print(correlation_matrix)
    
    # Visualización
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, vmin=-1, vmax=1, square=True, linewidths=1)
    plt.title('Matriz de Correlación entre Variables', fontsize=14, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    # Identificar correlaciones fuertes
    print("\n" + "="*70)
    print("CORRELACIONES FUERTES (|r| > 0.5):")
    print("="*70)
    
    strong_corr = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > 0.5:
                strong_corr.append({
                    'Variable 1': correlation_matrix.columns[i],
                    'Variable 2': correlation_matrix.columns[j],
                    'Correlación': correlation_matrix.iloc[i, j]
                })
    
    if strong_corr:
        strong_corr_df = pd.DataFrame(strong_corr).sort_values('Correlación', key=abs, ascending=False)
        print(strong_corr_df.to_string(index=False))
    else:
        print("No se encontraron correlaciones fuertes entre variables.")

## 4. Identificación de KPIs

Basándote en el análisis anterior, identifica los KPIs más relevantes para tu ONG.

In [ ]:
# Plantilla para documentar KPIs identificados
kpis_identificados = [
    {
        'nombre': 'Satisfacción del Beneficiario',
        'variable': 'satisfaccion',
        'tipo': 'Calidad del servicio',
        'valor_actual': df['satisfaccion'].mean() if 'satisfaccion' in df.columns else 0,
        'meta_propuesta': 8.5,
        'justificacion': 'Indicador directo de percepción de calidad por parte de beneficiarios'
    },
    {
        'nombre': 'Tiempo de Espera Promedio',
        'variable': 'tiempo_espera_min',
        'tipo': 'Eficiencia operativa',
        'valor_actual': df['tiempo_espera_min'].mean() if 'tiempo_espera_min' in df.columns else 0,
        'meta_propuesta': 15,  # minutos
        'justificacion': 'Mide eficiencia del proceso y respeto al tiempo de beneficiarios'
    },
    # Agrega más KPIs según tu análisis
]

print("KPIs IDENTIFICADOS")
print("="*70)
for i, kpi in enumerate(kpis_identificados, 1):
    print(f"\n{i}. {kpi['nombre']}")
    print(f"   Variable: {kpi['variable']}")
    print(f"   Tipo: {kpi['tipo']}")
    print(f"   Valor actual: {kpi['valor_actual']:.2f}")
    print(f"   Meta propuesta: {kpi['meta_propuesta']}")
    print(f"   Justificación: {kpi['justificacion']}")

# Crear DataFrame para exportar
kpis_df = pd.DataFrame(kpis_identificados)
print("\n" + "="*70)
print("RESUMEN DE KPIs")
print("="*70)
print(kpis_df[['nombre', 'valor_actual', 'meta_propuesta']].to_string(index=False))

## 5. Hallazgos Preliminares

Documenta los hallazgos principales de tu análisis.

In [ ]:
hallazgos = """
HALLAZGOS PRINCIPALES DEL ANÁLISIS EXPLORATORIO
{'='*70}

1. PERFIL DE BENEFICIARIOS:
   - [Describe características demográficas principales]
   - [Menciona si hay segmentos claramente diferenciados]

2. CALIDAD DEL SERVICIO:
   - [Nivel de satisfacción general]
   - [Variables que más impactan la satisfacción]
   - [Áreas de oportunidad identificadas]

3. DIFERENCIAS POR GRUPO:
   - [Si hay diferencias por género, ubicación, etc.]
   - [Cuáles grupos tienen mejor/peor experiencia]

4. KPIS PRIORITARIOS:
   - [Lista los 3-5 KPIs más importantes para esta ONG]
   - [Justifica por qué son prioritarios]

5. RECOMENDACIONES PRELIMINARES:
   - [Sugerencias basadas en los datos]
   - [Áreas que requieren atención inmediata]
"""

print(hallazgos)

# Opcional: Guardar hallazgos
# with open('hallazgos_preliminares.txt', 'w', encoding='utf-8') as f:
#     f.write(hallazgos)

## 6. Exportar Datos para Actividad #1 Parte 2

In [ ]:
# Crear resumen para incluir en tu entregable
resumen_actividad = f"""
ACTIVIDAD #1 PARTE 2: DIAGNÓSTICO ESTRATÉGICO
ANEXO: ANÁLISIS DE DATOS
{'='*70}

DATASET ANALIZADO:
- Número de registros: {len(df)}
- Variables analizadas: {len(df.columns)}
- Periodo de datos: [Completa con fechas si aplica]

KPIs IDENTIFICADOS:
"""

for i, kpi in enumerate(kpis_identificados, 1):
    resumen_actividad += f"""
{i}. {kpi['nombre']}
   - Valor actual: {kpi['valor_actual']:.2f}
   - Meta propuesta: {kpi['meta_propuesta']}
   - Justificación: {kpi['justificacion']}
"""

print(resumen_actividad)

# Guardar
# with open('resumen_analisis_datos.txt', 'w', encoding='utf-8') as f:
#     f.write(resumen_actividad)

## ✅ Checklist de Completitud

Antes de continuar, verifica que has:

- [ ] Cargado y revisado los datos de tu ONG
- [ ] Identificado y tratado valores faltantes
- [ ] Analizado variables demográficas principales
- [ ] Analizado variables de calidad del servicio
- [ ] Realizado análisis cruzado (satisfacción por grupos)
- [ ] Creado visualizaciones relevantes
- [ ] Identificado correlaciones importantes
- [ ] Documentado 3-5 KPIs prioritarios
- [ ] Redactado hallazgos preliminares
- [ ] Preparado resumen para Actividad #1 Parte 2

---

## 📚 Próximos Pasos

1. Completa tu **Actividad #1 Parte 2** con:
   - Diamante de Porter (ver `guias/guia_diamante_porter.md`)
   - Matriz BCG (ver `guias/guia_matriz_bcg.md`)
   - Customer Journey Map (ver `guias/guia_customer_journey_map.md`)
   - KPIs identificados en este análisis

2. **Semana 4:** Crearás visualizaciones con Looker Studio basándote en estos KPIs

---

**💡 TIP:** Guarda este notebook con tus análisis. Lo usarás en Semana 5 para el reporte final.
